# RL asmt 1

Discuss in detail how Markov Decision Process (MDP) can help in route planning application.
Justify your answer in 350-400 words. Explain in detail your environment, state space, action
space  and equations aligned with the given problem. Write the answer in the Colab Cell itself.


In [41]:
import pandas as pd
import numpy as np

In [59]:
# Reading the df and finding the prior, likelihood, predictor prior and posterior
df = pd.read_csv(r'C:\Users\praka\Acads\M_Tech\new_git_test\BITS-AIML-Mtech-V2\Sem_2\DRL\asmt1\data.csv')
df.head(25)

,Route,Time,Cost,Traffic Intensity,Selection
0,1,Best,Minimum,Medium,Yes
1,1,Average,Minimum,Low,Yes
2,1,Average,Maximum,Low,No
3,2,Average,Minimum,Medium,Yes
4,2,Best,Maximum,Medium,Yes
5,3,Best,Maximum,Low,No
6,3,Worst,Maximum,Medium,No
7,3,Best,Maximum,Low,Yes
8,4,Average,Minimum,High,No
9,4,Worst,Minimum,Medium,No


In [82]:
prior = df['Selection'].value_counts(normalize=True).round(6)  # class prior
predictor_prior = df['Route'].value_counts(normalize=True).round(6) # predictor prior
likelihood_no = df[df['Selection']=='No']['Route'].value_counts(normalize=True).round(6).sort_index() # likelihood
likelihood_yes = df[df['Selection']=='Yes']['Route'].value_counts(normalize=True).round(6).sort_index()


In [83]:
def find_posterior(class_likelihood, class_prior, pred_prior):
    tmp = class_likelihood * class_prior  # class posterior = class likelihood * class prior / predictor prior
    aligned_series1, aligned_series2 = tmp.align(pred_prior)
    return (aligned_series1/aligned_series2).round(6)

posterior_yes = find_posterior(likelihood_yes , prior['Yes'], predictor_prior).fillna(0)
posterior_no = find_posterior(likelihood_no , prior['No'], predictor_prior).fillna(0)

posterior_yes

Route
1    0.666665
2    1.000000
3    0.333332
4    0.000000
5    0.500000
6    1.000000
7    1.000000
8    0.500000
Name: proportion, dtype: float64

State transition probability P(St+1=S1 | St = S0, At=a).
In our case can be written as P(St+1=S1 | St = S0)  [State x State Matrix]

In [84]:
Pca_ = (posterior_yes[1]/(posterior_yes[1]+posterior_yes[2]+posterior_yes[3])).round(6)  # normalisation
Pda_ = (posterior_yes[3]/(posterior_yes[1]+posterior_yes[2]+posterior_yes[3])).round(6)
Pfa_ = (posterior_yes[2]/(posterior_yes[1]+posterior_yes[2]+posterior_yes[3])).round(6)

Pcf_ = (posterior_yes[4]/(posterior_yes[4]+posterior_yes[6])).round(6)
Pef_ = (posterior_yes[6]/(posterior_yes[4]+posterior_yes[6])).round(6)

state_transition_matrix = {
    'A' : {'A':0, 'B':0, 'C':Pca_, 'D':Pda_, 'E':0, 'F':Pfa_ },  # keys - initial, dict entries final
    'B' : {'A':0, 'B':0, 'C':0, 'D':0, 'E':0, 'F':0 },
    'C' : {'A':0, 'B':1, 'C':0, 'D':0, 'E':0, 'F':0 },
    'D' : {'A':0, 'B':0, 'C':0, 'D':0, 'E':1, 'F':0 },
    'E' : {'A':0, 'B':1, 'C':0, 'D':0, 'E':0, 'F':0 },
    'F' : {'A':0, 'B':0, 'C':Pcf_, 'D':0, 'E':Pef_, 'F':0 }
}

state_transition_matrix  # probability of transitioning to state x from state y

{'A': {'A': 0, 'B': 0, 'C': 0.333333, 'D': 0.166666, 'E': 0, 'F': 0.500001},
 'B': {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0},
 'C': {'A': 0, 'B': 1, 'C': 0, 'D': 0, 'E': 0, 'F': 0},
 'D': {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1, 'F': 0},
 'E': {'A': 0, 'B': 1, 'C': 0, 'D': 0, 'E': 0, 'F': 0},
 'F': {'A': 0, 'B': 0, 'C': 0.0, 'D': 0, 'E': 1.0, 'F': 0}}

Wherever there is only one path, we pick it so transition probability is 1.
Where there are multiple paths, we use normalised P(yes|route) to find state transition probabilities

In [85]:
# also keeping a mapping of inital_state->actions->final_state
state_action_mapping = {'A':{'R1':'C', 'R2':'F', 'R3':'D'},
                        'B':{},
                        'C':{'R7':'B'},
                        'D':{'R5':'E'},
                        'E':{'R8':'B'},
                        'F':{'R4':'C', 'R6':'E'}}

state_action_mapping

{'A': {'R1': 'C', 'R2': 'F', 'R3': 'D'},
 'B': {},
 'C': {'R7': 'B'},
 'D': {'R5': 'E'},
 'E': {'R8': 'B'},
 'F': {'R4': 'C', 'R6': 'E'}}

We will use E[Cost|RouteA] as Reward. Here cost can be switched with time taken or traffic or weighted average of these

In [86]:
time_mapping = {'Best':-1, 'Average':-2, 'Worst':-3  }
cost_mapping = {'Maximum':-1, 'Minimum':-2 }
traffic_mapping = {'High':-1, 'Medium':-2, 'Low':-3  }

df['Time'] = df['Time'].replace(time_mapping)
df['Cost'] = df['Cost'].replace(cost_mapping)
df['Traffic Intensity'] = df['Traffic Intensity'].replace(traffic_mapping)

df.head(15)

,Route,Time,Cost,Traffic Intensity,Selection
0,1,-1,-2,-2,Yes
1,1,-2,-2,-3,Yes
2,1,-2,-1,-3,No
3,2,-2,-2,-2,Yes
4,2,-1,-1,-2,Yes
5,3,-1,-1,-3,No
6,3,-3,-1,-2,No
7,3,-1,-1,-3,Yes
8,4,-2,-2,-1,No
9,4,-3,-2,-2,No


In [90]:
rewards = df.groupby('Route').agg({'Time':'mean', 'Cost':'mean', 'Traffic Intensity':'mean'})
reward_time = rewards['Time'].round(6)  # expected time based reward if action A is chosen
reward_cost = rewards['Cost'].round(6)  # same for traffic and cost
reward_traffic = rewards['Traffic Intensity'].round(6)
reward_combined =  (0.33*reward_time + 0.33*reward_cost + 0.33*reward_traffic).round(6) # considering all factors with equal weightage

reward_traffic

Route
1   -2.666667
2   -2.000000
3   -2.666667
4   -1.500000
5   -1.000000
6   -2.000000
7   -2.000000
8   -2.500000
Name: Traffic Intensity, dtype: float64

In [104]:
# Running value iteration script - time
def value_iteration(reward_type):
    num_iterations = 0
    value_vector = pd.Series({'A':1, 'B':1, 'C':1, 'D':1, 'E':1, 'F':1 }, name='value') # starting with an inital value estimate
    gamma = 1 # setting up a discounting factor
    while num_iterations < 15:
        updated_value = {}
        for state in ['A', 'B', 'C', 'D', 'E', 'F']:
            new_state_value = -10**12
            for action in range(1, 9):
                if 'R'+str(action) in list(state_action_mapping[state].keys()):
                    # action belongs to the state, find next state
                    next_state = state_action_mapping[state]['R'+str(action)]
                    next_state_transition_probability = state_transition_matrix[state][next_state]

                    new_state_value = max(new_state_value,
                                          reward_type[action] + gamma * next_state_transition_probability * value_vector[next_state] )
                else:
                    # this action dosent belongs to this state
                    pass

            #v_k+1 = max(R_a + gamma* P_a * v_k)
            updated_value[state] = value_vector[state] if new_state_value == -10**12 else new_state_value

        updated_value_vector = pd.Series(updated_value, name='value')
        # print('value vector at iteration {} is {}'.format(num_iterations, updated_value_vector))
        value_vector = updated_value_vector

        num_iterations +=1
        return value_vector


final_value_vector = value_iteration(reward_type=reward_time)
final_value_vector

A   -0.999999
B    1.000000
C   -1.500000
D   -1.500000
E    0.000000
F   -1.500000
Name: value, dtype: float64

In [105]:
final_value_vector = value_iteration(reward_type=reward_cost)
final_value_vector

A   -0.833334
B    1.000000
C   -1.000000
D    0.000000
E   -1.000000
F   -0.500000
Name: value, dtype: float64

In [106]:
final_value_vector = value_iteration(reward_type=reward_traffic)
final_value_vector

A   -1.499999
B    1.000000
C   -1.000000
D    0.000000
E   -1.500000
F   -1.000000
Name: value, dtype: float64

In [107]:
final_value_vector = value_iteration(reward_type=reward_combined)
final_value_vector

A   -1.149999
B    1.000000
C   -1.145000
D   -0.485000
E   -0.815000
F   -0.980000
Name: value, dtype: float64